<a href="https://colab.research.google.com/github/tiago-ferrari/ColisaoCirculos/blob/main/24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive'

MyDrive


In [ ]:
train_path = '/content/drive/MyDrive/JANEIRO 65/COLAB2/EMBRAPA REDUZIDO/train'
test_path = '/content/drive/MyDrive/JANEIRO 65/COLAB2/EMBRAPA REDUZIDO/valid'

In [ ]:
from PIL import Image 
import os 
from IPython.display import display
from IPython.display import Image as _Imgdis
# creating a object  

  

FileNotFoundError: ignored

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
vgg.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>

In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/JANEIRO 65/COLAB2/EMBRAPA REDUZIDO/train/*')
print(len(folders))

6


In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 732 images belonging to 6 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 631 images belonging to 6 classes.


In [ ]:
model.compile(loss='mse', optimizer='adam')

In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint



checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=2, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model_history=model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10

Epoch 00001: val_loss improved from inf to 0.22684, saving model to mymodel.h5
5/5 - 416s - loss: 0.2525 - val_loss: 0.2268 - 416s/epoch - 83s/step
Epoch 2/10
5/5 - 89s - loss: 0.2277 - 89s/epoch - 18s/step
Epoch 3/10
5/5 - 88s - loss: 0.1933 - 88s/epoch - 18s/step
Epoch 4/10
5/5 - 85s - loss: 0.1662 - 85s/epoch - 17s/step
Epoch 5/10
5/5 - 83s - loss: 0.1574 - 83s/epoch - 17s/step
Epoch 6/10
5/5 - 81s - loss: 0.1586 - 81s/epoch - 16s/step
Epoch 7/10
5/5 - 80s - loss: 0.1554 - 80s/epoch - 16s/step
Epoch 8/10
5/5 - 82s - loss: 0.1453 - 82s/epoch - 16s/step
Epoch 9/10
5/5 - 81s - loss: 0.1340 - 81s/epoch - 16s/step
Epoch 10/10
5/5 - 82s - loss: 0.1308 - 82s/epoch - 16s/step
Training completed in time:  0:22:21.856304


In [ ]:
_# Plot training & validation loss values
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('CNN Model accuracy values')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

KeyError: ignored

In [ ]:
print("b")

b


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/mymodel.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("mymodel.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpo76b_8_9/assets


59468808